# Predict Future Sales

This challenge serves as final project for the "How to win a data science competition" Coursera course. <br>
<br>
In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. 